In [1]:
from bs4 import BeautifulSoup
from tensorflow.python.platform import gfile
import urllib.request
import re
import json
import time

In [3]:
prefix = "http://www.ted.com"

In [4]:
def enlist_talk_names(path,dict_):
    print(path)
    req = urllib.request.Request(path, headers={'User-Agent': 'Mozilla/5.0'})
    r = urllib.request.urlopen(path).read()    
    soup = BeautifulSoup(r, "html5lib")
    videos_links = soup.find_all("div", class_="media__message")
    for element in videos_links:        
        dict_[element.a.get_text()] = {}
        dict_[element.a.get_text()]["link"] = prefix + element.a["href"]
                                         
    return dict_

In [5]:
all_talk_names={}
for i in range(1,75):
    path='https://www.ted.com/talks?page=%d'%(i)
    all_talk_names=enlist_talk_names(path,all_talk_names)        
    if i % 10 == 0:
        time.sleep(100)        

https://www.ted.com/talks?page=1
https://www.ted.com/talks?page=2
https://www.ted.com/talks?page=3
https://www.ted.com/talks?page=4
https://www.ted.com/talks?page=5
https://www.ted.com/talks?page=6
https://www.ted.com/talks?page=7
https://www.ted.com/talks?page=8
https://www.ted.com/talks?page=9
https://www.ted.com/talks?page=10
https://www.ted.com/talks?page=11
https://www.ted.com/talks?page=12
https://www.ted.com/talks?page=13
https://www.ted.com/talks?page=14
https://www.ted.com/talks?page=15
https://www.ted.com/talks?page=16
https://www.ted.com/talks?page=17
https://www.ted.com/talks?page=18
https://www.ted.com/talks?page=19
https://www.ted.com/talks?page=20
https://www.ted.com/talks?page=21
https://www.ted.com/talks?page=22
https://www.ted.com/talks?page=23
https://www.ted.com/talks?page=24
https://www.ted.com/talks?page=25
https://www.ted.com/talks?page=26
https://www.ted.com/talks?page=27
https://www.ted.com/talks?page=28
https://www.ted.com/talks?page=29
https://www.ted.com/tal

In [6]:
file_path = "/home/marchelo/MarcheloBragagnini/other/TED/videos.talks"
with gfile.GFile(file_path, mode="wb") as file:
    for _,data in all_talk_names.items():
        file.write(data['link'] + "\n")

In [7]:
def process_groups(src_ar, tgt_ar):
    src_ar = [0] + src_ar + [1e10]
    tgt_ar = [0] + tgt_ar + [1e10]
    
    src_res = [0]*len(src_ar)
    tgt_res = [0]*len(tgt_ar)
            
    src_pos = 1
    tgt_pos = 1
    it_src_ant = 0
    it_tgt_ant = 0
    group = 0
    while src_pos != len(src_ar) and tgt_pos != len(tgt_ar):
        if(src_ar[src_pos] == tgt_ar[tgt_pos]):
            src_res[it_src_ant:src_pos] = [group]*(src_pos-it_src_ant)
            tgt_res[it_tgt_ant:tgt_pos] = [group]*(tgt_pos-it_tgt_ant)
            group += 1
            it_src_ant = src_pos
            it_tgt_ant = tgt_pos
            src_pos += 1
            tgt_pos += 1
        elif src_ar[src_pos] < tgt_ar[tgt_pos] and src_pos != len(src_ar):
            src_pos += 1
        elif tgt_ar[tgt_pos] < src_ar[src_pos] and tgt_pos != len(tgt_ar):
            tgt_pos += 1
        
        if src_pos == len(src_ar) and tgt_pos == len(tgt_ar):
            if(group == -1):
                group += 1            
            src_res[it_src_ant:src_pos] = [group]*(src_pos-it_src_ant)
            tgt_res[it_tgt_ant:tgt_pos] = [group]*(tgt_pos-it_tgt_ant)            
    
    
    if src_res[1] != tgt_res[1]:
        it = 1
        if src_res[1] < tgt_res[1]:            
            while src_res[it] != tgt_res[1]:
                src_res[it] = tgt_res[1]
                it += 1
        else:
            while tgt_res[it] != src_res[1]:
                tgt_res[it] = src_res[1]
                it += 1
    
    return src_res[1:-1], tgt_res[1:-1]

In [12]:
def get_subtitles_parallel(id_talk, src_lang, tgt_lang):
    url_trans_src = "https://www.ted.com/talks/" + id_talk + "/transcript.json?language=" + src_lang
    url_trans_tgt = "https://www.ted.com/talks/" + id_talk + "/transcript.json?language=" + tgt_lang
  
    try:
        src_file = urllib.request.urlopen(url_trans_src)
        tgt_file = urllib.request.urlopen(url_trans_tgt)
        data_tgt = json.loads(tgt_file.read().decode())
        data_src = json.loads(src_file.read().decode())        
    except urllib.error.HTTPError as e:
        return [], []
    
    '''
    # metodo menos preciso
    if len(data_src['paragraphs']) != len(data_tgt['paragraphs']):
        return ar_text_src, ar_text_tgt
                            
    for para_src, para_tgt in zip(data_src['paragraphs'],data_tgt['paragraphs']):
        sente_src = ' '.join(it['text'] for it in para_src['cues'])
        sente_src = re.sub('\n|\r', ' ', sente_src)
        ar_text_src.append(sente_src)
        
        sente_tgt = ' '.join(it['text'] for it in para_tgt['cues'])
        sente_tgt = re.sub('\n|\r', ' ', sente_tgt)
        ar_text_tgt.append(sente_tgt)
    '''    
    
    ar_text_src, ar_text_tgt = [],[]
    src_dict_sent, tgt_dict_sent = {},{}
    src_ar_time, tgt_ar_time = [],[]
    for para_src, para_tgt in zip(data_src['paragraphs'], data_tgt['paragraphs']):
        for it in para_src['cues']:
            text = it['text']
            text = re.sub('\n|\r', ' ', text)
            src_dict_sent[int(it['time'])] = text
            src_ar_time.append(int(it['time']))
        for it in para_tgt['cues']:
            text = it['text']
            text = re.sub('\n|\r', ' ', text)
            tgt_dict_sent[int(it['time'])] = text
            tgt_ar_time.append(int(it['time']))
            
    src_group_time, tgt_group_time = process_groups(src_ar_time, tgt_ar_time)
    init_gr, end_gr = src_group_time[0], src_group_time[-1]            
    src_pos, tgt_pos = 0, 0

    while init_gr <= end_gr:
        text_src, text_tgt = '',''
        while src_pos != len(src_ar_time) and src_group_time[src_pos] == init_gr:                                    
            text_src = text_src + ' ' + src_dict_sent[src_ar_time[src_pos]]            
            src_pos += 1
        while tgt_pos != len(tgt_ar_time) and tgt_group_time[tgt_pos] == init_gr:                        
            text_tgt = text_tgt + ' ' + tgt_dict_sent[tgt_ar_time[tgt_pos]]
            tgt_pos += 1
        ar_text_src.append(text_src)
        ar_text_tgt.append(text_tgt)
        init_gr += 1
                
    #print('\n\n'.join("%s:%s\n%s:%s"%(src_lang,it_src,tgt_lang,it_tgt) for it_src, it_tgt in zip(ar_text_src, ar_text_tgt)))
    return ar_text_src, ar_text_tgt


In [8]:
def write_parallel_corpus(array_text, path_file):    
    with gfile.GFile(path_file, mode="ab") as f:        
        for sente in array_text:
            f.write(sente + '\n')

In [9]:
src_lang = 'es'
tgt_lang = 'fr'

In [10]:
path_src_file = "/home/marchelo/MarcheloBragagnini/dataSets/TED/spanish-french/ted." + src_lang
path_tgt_file = "/home/marchelo/MarcheloBragagnini/dataSets/TED/spanish-french/ted." + tgt_lang

In [13]:
# process all TED talks
n_sent_src, n_sent_tgt = 0,0
for name_vid, data in all_talk_names.items():    
    req_talk = urllib.request.Request(data['link'], headers={'User-Agent': 'Mozilla/5.0'})
    r_talk = urllib.request.urlopen(req_talk).read()
    soup_talk = BeautifulSoup(r_talk, "html5lib")
    meta_ = soup_talk.find_all('meta', property='al:ios:url')
    id_talk = re.findall(r'\d+', meta_[0].attrs['content'])[0]
        
    ar_text_src, ar_text_tgt = get_subtitles_parallel(id_talk, src_lang, tgt_lang)
    if len(ar_text_src) != 0:    
        write_parallel_corpus (ar_text_src, path_src_file)
        write_parallel_corpus (ar_text_tgt, path_tgt_file)
        n_sent_src += len(ar_text_src)
        n_sent_tgt += len(ar_text_tgt)
        
    print(name_vid[0:-2] + " (%d,%d)"%(len(ar_text_src), len(ar_text_tgt)))
        
print("language source(%s):%d"%(src_lang, n_sent_src))
print("language target(%s):%d"%(tgt_lang, n_sent_tgt))



The birth of virtual reality as an art for (260,260)

Physics is fun to imagin (0,0)

Refugees have the right to be protecte (1,1)

Political change with pen and pape (150,150)

A multimedia theatrical adventur (127,127)

A Darwinian theory of beaut (364,364)

In search of the man who broke my nec (360,360)

Is anatomy destiny (478,478)

What's hidden under the Greenland ice sheet (25,25)

Great cars are great ar (360,360)

The levitating superconducto (0,0)

The secret US prisons you've never heard of befor (0,0)

Time to end the war in Afghanista (0,0)

Bring on the learning revolution (0,0)

The fascinating physics of everyday lif (0,0)

How to use experts — and when not t (0,0)

The surprising habits of original thinker (0,0)

Why Libya's revolution didn't work — and what migh (0,0)

The clues to a great stor (0,0)

The bio-future of joint replacemen (0,0)

What does my headscarf mean to you (0,0)

A guerilla gardener in South Central L (0,0)

The case for engineering our foo (0,0


Designers — think big (0,0)

Growing evidence of brain plasticit (0,0)

The good news about PM (0,0)

Being young and making an impac (0,0)

How to run a company with (almost) no rule (0,0)

How augmented reality will change sports ... and build empath (0,0)

The fight against sex slaver (0,0)

A song for my hero, the woman who rowed into a hurrican (86,86)

Design at the intersection of technology and biolog (0,0)

What's next in 3D printin (7,7)

What are animals thinking and feeling (0,0)

Conception to birth — visualize (0,0)

Planning for the end of oi (0,0)

Energy from floating algae pod (0,0)

The case for anonymity onlin (259,259)

The music war (0,0)

Why I'm done trying to be "man enough (0,0)

A family tree for humanit (0,0)

TED's secret to great public speakin (0,0)

3 stories of local eco-entrepreneurshi (0,0)

The sibling bon (0,0)

My love letter to cospla (0,0)

Teach girls bravery, not perfectio (0,0)

Teaching kids real math with computer (0,0)

A global culture to


5 dangerous things you should let your kids d (158,158)

Dreams from endangered culture (0,0)

The 3 A's of awesom (0,0)

Cape Breton fiddling in reel tim (0,0)

Don't take consciousness for grante (0,0)

How to combat modern slaver (0,0)

How the Net aids dictatorship (0,0)

"Tembererana (153,153)

Humble plants that hide surprising secret (0,0)

Wearing nothing ne (125,125)

The intricate economics of terroris (0,0)

Africa's cheetahs versus hippo (0,0)

A visual history of human knowledg (0,0)

My father, locked in his body but soaring fre (0,0)

Demo: The Or (0,0)

Trash cart superheroe (0,0)

How germs travel on planes — and how we can stop the (0,0)

A powerful poem about what it feels like to be transgende (0,0)

Dancing with ligh (8,8)

For more tolerance, we need more ... tourism (9,9)

A next-generation digital boo (0,0)

Mining minerals from seawate (73,73)

How do you build a sacred space (63,63)

Can art amend history (0,0)

The future of flying robot (0,0)

How to defend


Teen wonders play bluegras (0,0)

The Blur Building and other tech-empowered architectur (0,0)

A musical escape into a world of light and colo (0,0)

We can reprogram life. How to do it wisel (0,0)

Hands-on science with squishy circuit (0,0)

The idea behind Zipcar (and what comes next (0,0)

The biggest risks facing cities — and some solution (0,0)

The danger of silenc (3,3)

Sculptures that’d be at home in the deep se (12,12)

Fifty shades of ga (0,0)

Using tech to enable dreamin (157,157)

The danger of science denia (411,411)

The Housing First approach to homelessnes (0,0)

The line between life and not-lif (402,402)

How an obese town lost a million pound (0,0)

Pay attention to penguin (0,0)

Fashion has a pollution problem — can biology fix it (0,0)

The simple power of hand-washin (0,0)

Strange answers to the psychopath tes (358,358)

Unseen footage, untamed natur (0,0)

The extraordinary power of ordinary peopl (0,0)

Inspiring a life of immersio (0,0)

See invisible mo


Let's design social media that drives real chang (0,0)

Nature. Beauty. Gratitude (0,0)

The key to growth? Race with the machine (0,0)

An animated tour of the invisibl (0,0)

Metal that breathe (161,161)

Taking imagination seriousl (205,205)

Fractals and the art of roughnes (0,0)

The sore problem of prosthetic limb (0,0)

My mother’s strange definition of empowermen (0,0)

East vs. West — the myths that mystif (0,0)

Transition to a world without oi (0,0)

Making peace is a maratho (0,0)

The mathematics of wa (0,0)

Superheroes inspired by Isla (0,0)

The price of sham (0,0)

A third way to think about ai (373,373)

How to make stress your frien (4,4)

My mushroom burial sui (0,0)

The debut of the British Paraorchestr (0,0)

Got a meeting? Take a wal (0,0)

The global learning crisis — and what to do about i (0,0)

Brain-to-brain communication has arrived. How we did i (0,0)

Could a drug prevent depression and PTSD (0,0)

Machine intelligence makes human morals more importan (

...


Art that craves your attentio (0,0)

A country with no wate (0,0)

My wish: Once Upon a Schoo (0,0)

An invitation to men who want a better world for wome (0,0)

What I saw in the wa (209,209)

A monkey economy as irrational as our (504,504)

Should we simplify spelling (0,0)

Let the environment guide our developmen (0,0)

Two poems about what dogs think (probably (0,0)

Living beyond limit (0,0)

The price of happines (289,289)

The nerd's guide to learning everything onlin (224,224)

Architecture that's built to hea (10,10)

What if our healthcare system kept us healthy (412,412)

3 lessons on success from an Arab businesswoma (128,128)

Things I've learned in my life so fa (0,0)

A virtual choir 2,000 voices stron (0,0)

Why I chose a gu (0,0)

The ocean's shifting baselin (207,207)

How to live before you di (0,0)

Happiness by desig (0,0)

A plea for bee (0,0)

The security mirag (0,0)

What a planet needs to sustain lif (0,0)

How social media can make histor (0,0)

Can we buil


The four fish we're overeating — and what to eat instea (1,1)

A doctor's touc (0,0)

Coding a better governmen (289,289)

Do what you love (no excuses! (0,0)

To This Day ... for the bullied and beautifu (0,0)

What we learned from teetering on the fiscal clif (381,381)

Where to train the world's doctors? Cuba (0,0)

Who are you, really? The puzzle of personalit (0,0)

Why jobs of the future won't feel like wor (143,143)

Why good hackers make good citizen (0,0)

The risky politics of progres (0,0)

This is what enduring love looks lik (0,0)

Remember to say thank yo (80,80)

Forget the pecking order at wor (0,0)

Should you donate differently (0,0)

I leapt from the stratosphere. Here's how I did i (0,0)

The hunt for a supermassive black hol (0,0)

Meet Spot, the robot dog that can run, hop and open door (0,0)

How cohousing can make us happier (and live longer (0,0)

3 principles for creating safer A (0,0)

Why the only future worth building includes everyon (112,112)

How a vide


The unexpected benefit of celebrating failur (0,0)

It’s our city. Let’s fix i (0,0)

The transformative power of classical musi (0,0)

Building a dinosaur from a chicke (372,372)

How an old loop of railroads is changing the face of a cit (0,0)

Older people are happie (0,0)

Flip your thinking on AIDS in Afric (0,0)

My road trip through the whitest towns in Americ (0,0)

The day I turned down Tim Berners-Le (0,0)

A robot that runs and swims like a salamande (0,0)

"Kiteflyer's Hill (0,0)

How to escape education's death valle (375,375)

What our language habits revea (393,393)

Bringing peace to the minds of Afghanista (0,0)

The world is one big dataset. Now, how to photograph it .. (0,0)

Let's save the last pristine continen (0,0)

An electrifying acoustic guitar performanc (0,0)

Trust, morality — and oxytocin (397,397)

The history of human emotion (0,0)

Odes to vice and consequence (298,298)

Poetry of youth and ag (0,0)

Keep your goals to yoursel (0,0)

A bath without wat


4 lessons from robots about being huma (0,0)

How Christmas lights helped guerrillas put down their gun (6,6)

And for my next trick, a robo (0,0)

Let's talk about dyin (0,0)

A glimpse of life on the roa (14,14)

A new strategy in the war on cance (434,434)

How architecture can connect u (0,0)

It's time for "The Talk (125,125)

What we learn before we're bor (0,0)

A provocative way to finance the fight against climate chang (0,0)

Comedy is translatio (0,0)

The mysterious workings of the adolescent brai (291,291)

How I taught rats to sniff out land mine (0,0)

DJ decks made of ... pape (7,7)

"St. James Infirmary Blues (0,0)

Architecture at home in its communit (0,0)

Got a wicked problem? First, tell me how you make toas (0,0)

The world needs all kinds of mind (0,0)

How I swam the North Pol (0,0)

Why we shouldn't trust markets with our civic lif (0,0)

Our refugee system is failing. Here's how we can fix i (0,0)

Why I must speak out about climate chang (0,0)

How we unear


What do we do when antibiotics don't work any more (0,0)

Lessons from past president (0,0)

A temporary tattoo that brings hospital care to the hom (138,138)

Stunning photos of the endangered Everglade (68,68)

A message to gay teens: It gets bette (0,0)

The art of wearable communicatio (200,200)

Why we laug (197,197)

4 lessons in creativit (0,0)

The brain may be able to repair itself — with hel (0,0)

My wish: Use art to turn the world inside ou (0,0)

What we don't know about mother's mil (0,0)

A smarter, more precise way to think about public healt (0,0)

World-class health car (0,0)

The thrilling potential of SixthSense technolog (0,0)

This telescope might show us the beginning of the univers (0,0)

The single biggest health threat women fac (0,0)

A bold plan to house 100 million peopl (0,0)

Save the oceans, feed the world (0,0)

A leap from the edge of spac (0,0)

What's so funny about mental illness (150,150)

The secret structure of great talk (0,0)

If a story moves